# Train
I did not change anything here except number of epochs. Is there any reason to tune anything?
Keras automatically resizes images, so no need to worry about that.

In [1]:
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import matplotlib
import keras.backend as K
matplotlib.style.use('ggplot')

2023-02-26 19:30:19.483176: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = 'training/'
VALID_DATA_DIR = 'validation/'

In [3]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)
train_generator = datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    shuffle=True,
    target_size=IMAGE_SHAPE,
)
valid_generator = datagen.flow_from_directory(
    VALID_DATA_DIR,
    shuffle=False,
    target_size=IMAGE_SHAPE,
)

Found 848 images belonging to 2 classes.
Found 210 images belonging to 2 classes.


In [4]:
def build_model(num_classes):
    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', 
                           input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model
model = build_model(num_classes=2)

2023-02-26 19:30:32.615748: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def get_f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    
    return f1_val

In [6]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[get_f1]
)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 16)      2320      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 16)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 32)        4640      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 32)       0

In [7]:
EPOCHS = 10
BATCH_SIZE = 16
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // BATCH_SIZE // 2,
                    epochs=EPOCHS,
                    validation_data=valid_generator,
                    validation_steps= valid_generator.samples // BATCH_SIZE // 2,
                    verbose=1
                    )

Epoch 1/10
26/26 [==============================] - 42s 1s/step - loss: 0.6803 - get_f1: 0.5661 - val_loss: 0.6755 - val_get_f1: 0.5417
Epoch 2/10
26/26 [==============================] - 36s 1s/step - loss: 0.6503 - get_f1: 0.5853 - val_loss: 0.6344 - val_get_f1: 0.6094
Epoch 3/10
26/26 [==============================] - 39s 1s/step - loss: 0.6048 - get_f1: 0.6058 - val_loss: 0.5902 - val_get_f1: 0.6458
Epoch 4/10
26/26 [==============================] - 46s 2s/step - loss: 0.5768 - get_f1: 0.6466 - val_loss: 0.5556 - val_get_f1: 0.6667
Epoch 5/10
26/26 [==============================] - 38s 1s/step - loss: 0.5334 - get_f1: 0.6875 - val_loss: 0.5480 - val_get_f1: 0.6667
Epoch 6/10
26/26 [==============================] - 35s 1s/step - loss: 0.4979 - get_f1: 0.7007 - val_loss: 0.4919 - val_get_f1: 0.6927
Epoch 7/10
26/26 [==============================] - 48s 2s/step - loss: 0.4888 - get_f1: 0.6995 - val_loss: 0.4803 - val_get_f1: 0.7292
Epoch 8/10
26/26 [==============================

In [9]:
model.save_weights('igor_alentev')